In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

20:41:20 INFO:Extracting Amazon-Google...
20:41:20 INFO:Reading Amazon-Google record_dict...
20:41:20 INFO:Reading Amazon-Google train.csv...
20:41:20 INFO:Reading Amazon-Google valid.csv...
20:41:20 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'manufacturer', 'price']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed.data_utils.field_config_parser import DEFAULT_ALPHABET

alphabet = DEFAULT_ALPHABET
field_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'title_semantic': {
        'key': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [9]:
from entity_embed import FieldConfigDictParser

record_numericalizer = FieldConfigDictParser.from_dict(
    field_config_dict, record_list=benchmark.record_dict.values())

20:41:21 INFO:For field=title, computing actual max_str_len
20:41:21 INFO:For field=title, using actual_max_str_len=26
20:41:21 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
20:41:24 INFO:For field=manufacturer, computing actual max_str_len
20:41:24 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
20:41:24 INFO:For field=manufacturer, using actual_max_str_len=16
20:41:24 INFO:For field=price, computing actual max_str_len
20:41:24 INFO:actual_max_str_len=9 must be even to enable NN pooling. Updating to 10
20:41:24 INFO:For field=price, using actual_max_str_len=10


In [10]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_datamodule(
    record_numericalizer=record_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [11]:
from entity_embed import Matcher

model = Matcher(
    record_numericalizer=record_numericalizer
)

In [ ]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

20:41:24 INFO:GPU available: True, used: True
20:41:24 INFO:TPU available: False, using: 0 TPU cores
20:41:24 INFO:Train positive pair count: 699
20:41:24 INFO:Train negative pair count: 6175
20:41:24 INFO:Valid positive pair count: 234
20:41:24 INFO:Valid positive pair count: 2059
20:41:24 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
20:41:41 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 11.2 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
9.5 M     Trainable params
1.7 M     Non-trainable params
11.2 M    Total params
44.641    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model.validate(datamodule)

## Testing

In [ ]:
model.test(datamodule)